<a href="https://colab.research.google.com/github/Anderson-Andre-P/Summary-Generator-With-Python-and-NLTK/blob/main/pdf_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 6.3 MB/s eta 0:00:00


In [2]:
!pip install reportlab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 84.3 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


In [1]:
# Libs para upload de arquivo
from google.colab import files

# Libs para sumarização
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import heapq
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer

# Libs para extração de palavras
import PyPDF2
from PyPDF2 import PdfReader

# Libs para limpeza de strings
import re

# Libs para geração de PDF
import io
from reportlab.pdfgen import canvas
import textwrap

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
uploaded = files.upload()

Saving Resenha Artigo - Anderson André.pdf to Resenha Artigo - Anderson André (1).pdf


In [3]:
# Abrindo o arquivo PDF enviado
pdf_file = open('/content/Resenha Artigo - Anderson André.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)

In [4]:
pdf_text = ""

In [5]:
# Lendo o arquivo PDF enviado e extraindo as palavras
reader = PdfReader('/content/Resenha Artigo - Anderson André.pdf')
for page in reader.pages:
    pdf_text += page.extract_text()
    # print(page.extract_text())

In [6]:
# Quantidade de páginas do PDF
print(f"pdf page count : {len(reader.pages)}")

pdf page count : 3


In [7]:
# Juntando as palavras em uma variável
pdf_text = ' '.join(pdf_text.split())

In [8]:
print(pdf_text)

Nome: Anderson André Pereira Eleutério; RA: 015642. 1. Introdução Segundo Fadel e Silveira (2010), as metodologias ágeis são muito utilizadas por empresas desenvolvedoras de software que visam administrar melhor seus recursos e o tempo disponível para desenvolvimento das atividades. Um dos objetivos que as organizações têm ao utilizar as metodologias ágeis é melhorar o fluxo do desenvolvimento de software, alocando menos recursos na produção e aumentando a eficiência das equipes, isso melhora a qualidade dos produtos desenvolvidos. Os mesmos autores, informam que o um encontro realizado por 17 desenvolvedores de software identificou 12 princípios e 4 premissas que estão presentes no manifesto ágil. As 4 premissas podem ser representados nos seguintes tópicos: ● Valorizar pessoas e interações acima de processos e ferramentas; ● Priorizar o software funcional em vez de documentação detalhada; ● Dar importância à colaboração com o cliente em vez de negociação de contratos; ● Ser adaptável

In [9]:
# Removendo tudo que não é alphanumerico
pdf_text = re.sub(r'[^\w\s]', '', pdf_text)
# re.sub('[^A-Za-z0-9]+', ' ', pdf_text)

In [10]:
# Removendo todos os números
pdf_text = re.sub(r'[0-9]', ' ', pdf_text)

# Removendo espaços duplicados
pdf_text = re.sub(r' +', ' ', pdf_text)

# # Removendo palavras específicas
# pdf_text = re.sub(r'\Disponível em', ' ', pdf_text)
# pdf_text = re.sub(r'\Acesso em', ' ', pdf_text)

In [11]:
# Tokenizando o texto em sentenças e palavras
sentencas = sent_tokenize(pdf_text)
palavras = word_tokenize(pdf_text.lower())

In [12]:
# Removendo as stopwords e pontuações
stop_words = set(stopwords.words('portuguese'))
palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stop_words and palavra.isalpha()]

In [13]:
# Aplicando o stemmer para reduzir as palavras para sua raiz
ps = PorterStemmer()
palavras_stemmed = [ps.stem(palavra) for palavra in palavras_sem_stopwords]

In [14]:
# Criando um dicionário de frequência de palavras
frequencia_palavras = nltk.FreqDist(palavras_stemmed)

In [15]:
# Calculando a pontuação de cada sentença com base nas frequências das palavras
pontuacao_sentencas = {}
for sentenca in sentencas:
    for palavra in word_tokenize(sentenca.lower()):
        if palavra in palavras_sem_stopwords:
            if sentenca not in pontuacao_sentencas:
                pontuacao_sentencas[sentenca] = frequencia_palavras[ps.stem(palavra)]
            else:
                pontuacao_sentencas[sentenca] += frequencia_palavras[ps.stem(palavra)]

In [16]:
# Selecionando as melhores sentenças com base nas maiores pontuações
melhores_sentencas = heapq.nlargest(100, pontuacao_sentencas, key=pontuacao_sentencas.get)
resumo = ' '.join(melhores_sentencas)

In [17]:
# Imprimindo o resumo gerado
print("Resumo: ", resumo)

Resumo:  Nome Anderson André Pereira Eleutério RA Introdução Segundo Fadel e Silveira as metodologias ágeis são muito utilizadas por empresas desenvolvedoras de software que visam administrar melhor seus recursos e o tempo disponível para desenvolvimento das atividades Um dos objetivos que as organizações têm ao utilizar as metodologias ágeis é melhorar o fluxo do desenvolvimento de software alocando menos recursos na produção e aumentando a eficiência das equipes isso melhora a qualidade dos produtos desenvolvidos Os mesmos autores informam que o um encontro realizado por desenvolvedores de software identificou princípios e premissas que estão presentes no manifesto ágil As premissas podem ser representados nos seguintes tópicos Valorizar pessoas e interações acima de processos e ferramentas Priorizar o software funcional em vez de documentação detalhada Dar importância à colaboração com o cliente em vez de negociação de contratos Ser adaptável às mudanças em vez de seguir um plano in

In [18]:
def adicionar_quebras_de_linha(texto, caracteres_por_linha=75):
    linhas = textwrap.wrap(resumo, caracteres_por_linha)
    return "\n".join(linhas)

def criar_pdf(arquivo_pdf=reader, resumo=resumo, caracteres_por_linha=70):
    buffer = io.BytesIO()
    pdf_canvas = canvas.Canvas(buffer)

    paginas_do_resumo = textwrap.wrap(resumo, caracteres_por_linha)

    # linhas = texto_com_quebras_de_linha.split('\n')

    x, y = 100, 750

    # desenha cada linha em uma posição diferente no canvas
    for indice_pagina, pagina in enumerate(paginas_do_resumo):
        # pdf_canvas.drawText(x, y, linha)
        pdf_canvas.drawString(x, y, pagina)
        y -= 15  # ajusta a posição da próxima linha
        if indice_pagina < len(paginas_do_resumo) - 1 and y <= 100:
            pdf_canvas.showPage()
            y = 750

    pdf_canvas.save()

    # salva o conteúdo do objeto buffer em um arquivo PDF
    with open("resumo_gerado.pdf", "wb") as output_file:
        output_file.write(buffer.getvalue())

criar_pdf()

In [19]:
files.download('resumo_gerado.pdf') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>